In [ ]:
'''
Project Euler Problem 084: Monopoly Odds
'''

In [ ]:
from random import randint, shuffle

In [ ]:
squares = ['GO', 'A1', 'CC1', 'A2', 'T1', 'R1', 'B1', 'CH1', 'B2', 'B3',
           'JAIL', 'C1', 'U1', 'C2', 'C3', 'R2', 'D1', 'CC2', 'D2', 'D3',
           'FP', 'E1', 'CH2', 'E2', 'E3', 'R3', 'F1', 'F2', 'U2', 'F3',
           'G2J', 'G1', 'G2', 'CC3', 'G3', 'R4', 'CH3', 'H1', 'T2', 'H2']

n_squares = len (squares)

squares_visited = {sq: 0 for sq in squares}

chest_squares = ['CC1', 'CC2', 'CC3']
chance_squares = ['CH1', 'CH2', 'CH3']

pos_railroad = [squares.index(sq) for sq in ('R1', 'R2', 'R3', 'R4')]
pos_utility = [squares.index(sq) for sq in ('U1', 'U2')]

chest = ['GO', 'JAIL'] + [''] * 14
n_chest = len(chest)

chance = ['GO', 'JAIL', 'C1', 'E3', 'H2', 'R1', 'R', 'R', 'U', '-3'] + [''] * 6
n_chance = len(chance)

go = 'GO'
g2j = 'G2J'
jail = 'JAIL'

In [ ]:
def move(roll: int) -> None:
    global pos, pos_chest, pos_chance, n_chest, n_chance

    pos = (pos+roll) % n_squares
    square = squares[pos]
    # print(square)

    if square == g2j:
        # print('go to jail')
        pos = squares.index(jail)
        squares_visited[jail] += 1
        return

    if square in chance_squares:
        # print('chance', end=' ')
        ch = chance[pos_chance]
        match ch:
            case '':
                # print('stay')
                squares_visited[square] += 1
            case 'G2J':
                # print('go to jail')
                pos = squares[jail]
                squares_visited[jail] += 1
            case '-3':
                # print('go back to', end=' ')
                pos = (pos - 3) % n_squares
                squares_visited[squares[pos]] += 1
                # print(squares[pos])
            case 'R':
                for p in pos_railroad:
                    if pos < p:
                        pos = p
                        squares_visited[squares[pos]] += 1
                        pos_chance = (pos_chance + 1) % n_chance
                        # print(f'go to railroad {squares[pos]} (inner)')
                        return
                pos = pos_railroad[0]
                squares_visited[squares[pos]] += 1
                # print(f'go to railroad {squares[pos]} (outer)')

            case 'U':
                if pos < (p:=pos_utility[0]):
                    pos = p
                    squares_visited[squares[pos]] += 1
                else:
                    pos = pos_utility[1]
                    squares_visited[squares[pos]] += 1
                # print(f'go to utility {squares[pos]}')
            case _:
                pos = squares.index(ch)
                squares_visited[ch] += 1
                # print(f'go to {squares[pos]}')

        pos_chance = (pos_chance + 1) % n_chance
        return

    if square in chest_squares:
        # print('chest', end=' ')
        ch = chest[pos_chest]
        match ch:
            case '':
                # print('stay')
                squares_visited[square] += 1
            case 'G2J':
                # print('go to jail')
                pos = squares[jail]
                squares_visited[jail] += 1
            case _:
                # print(f'go to {square}')
                pos = squares.index(ch)
                squares_visited[ch] += 1

        pos_chest = (pos_chest + 1) % n_chest
        return
    
    squares_visited[square] += 1


In [ ]:
n_turns = 5_000_000
sides = 4
squares_visited = {sq: 0 for sq in squares}

shuffle(chest)
shuffle(chance)

pos = 0
pos_chest = pos_chance = 0

count_doubles = 0
total_doubles = 0

for i in range(n_turns):
    # print(i, end=' ')
    roll1, roll2 = randint(1, sides), randint(1, sides)

    if roll1 == roll2:
        total_doubles += 1
        count_doubles += 1
        if count_doubles == 3:
            count_doubles = 0
            pos = squares.index(jail)
            squares_visited[jail] += 1
            continue
        else:
            move(roll1+roll2)

    else:
        count_doubles = 0
        move(roll1+roll2)

# print(f'Jail: {squares_visited[jail]}')
# print(f'G2J: {squares_visited[g2j]}')
# print(f'GO: {squares_visited[go]}')

squares_ranked = sorted([(k, v/n_turns) for k, v in squares_visited.items()], key=lambda x: x[1], reverse=True)

result = ''.join([f'{squares.index(sq[0]):02}' for sq in squares_ranked[:3]])

print(f'The six-digit modal string for {sides}-sided dice is {result}')

In [ ]:
squares_ranked